In [1]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# MapBox token
token = "pk.eyJ1IjoiZnlubndpIiwiYSI6ImNrODk3YmF6MzAzcDczbWs5NXdhaGpyNzYifQ.vHweJb-1hjDeE21tTs7tGQ"

## Load data

In [2]:
data = pd.read_pickle("final_data/temperature_summary.pkl")
stations = pd.read_

FileNotFoundError: [Errno 2] No such file or directory: 'final_data/temperature_summary.pkl'